In [99]:
import numpy as np
import pandas as pd
import math

#import import_ipynb
import nbimporter
from Scoring_Function import score

# Read csv into pandas dataframe
df = pd.read_csv('train_fixed.csv', low_memory=False)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [100]:
plays = pd.unique(df['PlayId'])
plays

array([20170907000118, 20170907000139, 20170907000189, ...,
       20181230154082, 20181230154135, 20181230154157])

In [101]:
A_id = 20170907000118
B_id = 20170907000139

In [102]:
def calc_similar(A_id, B_id):
    A = df[df['PlayId'] == 20170907000118]
    B = df[df['PlayId'] == 20170907000139]
    
    # Set up Play A for matching
    players_A = pd.DataFrame(columns = A.columns)
    count_A = 0
    rush_index_A = 0
    
    for i in range(len(A)):
        count_A += 1
        if A.iloc[i]['NflId'] == A.iloc[i]['NflIdRusher']:
            rush_index_A = count_A
            rusher_A = pd.DataFrame(columns = A.columns)
            rusher_A = rusher_A.append(A.iloc[i])
        else:
            players_A = players_A.append(A.iloc[i])
            
    if rush_index_A < 12:
        offense_A = players_A[:10]
        defense_A = players_A[10:]
    else:
        offense_A = players_A[11:]
        defense_A = players_A[:11]
        
    offense_A = offense_A[offense_A['Position'].isin(['WR', 'TE', 'RB', 'HB'])]
    
    # Set up Play B for matching
    players_B = pd.DataFrame(columns = B.columns)
    count_B = 0
    rush_index_B = 0
    
    for i in range(len(B)):
        count_B += 1
        if B.iloc[i]['NflId'] == B.iloc[i]['NflIdRusher']:
            rush_index_B = count_B
            rusher_B = pd.DataFrame(columns = B.columns)
            rusher_B = rusher_B.append(B.iloc[i])
        else:
            players_B = players_B.append(B.iloc[i])
            
    if rush_index_B < 12:
        offense_B = players_B.iloc[:10]
        defense_B = players_B.iloc[10:]
    else:
        offense_B = players_B.iloc[11:]
        defense_B = players_B.iloc[:11]
        
    offense_B = offense_B[offense_B['Position'].isin(['WR', 'TE', 'RB', 'HB'])]
    
    # Check if offensive personnel amount is matching
    if len(offense_A) != len(offense_B):
        return 1000 # Return large error
    
    # Find players' positions relative to the rusher for Play A
    if offense_A['PlayDirection'].iloc[0] == 'left':
        offense_A['X_scaled'] = offense_A['X'] - rusher_A['X'].iloc[0]
        defense_A['X_scaled'] = defense_A['X'] - rusher_A['X'].iloc[0]
    else:
        offense_A['X_scaled'] = rusher_A['X'].iloc[0] - offense_A['X']
        defense_A['X_scaled'] = rusher_A['X'].iloc[0] - defense_A['X']
        
    offense_A['Y_scaled'] = rusher_A['Y'].iloc[0] - offense_A['Y']
    defense_A['Y_scaled'] = rusher_A['Y'].iloc[0] - defense_A['Y']
    
    # Find players' positions relative to the rusher for Play B
    if offense_B['PlayDirection'].iloc[0] == 'left':
        offense_B['X_scaled'] = offense_B['X'] - rusher_B['X'].iloc[0]
        defense_B['X_scaled'] = defense_B['X'] - rusher_B['X'].iloc[0]
    else:
        offense_B['X_scaled'] = rusher_B['X'].iloc[0] - offense_B['X']
        defense_B['X_scaled'] = rusher_B['X'].iloc[0] - defense_B['X']
        
    offense_B['Y_scaled'] = rusher_B['Y'].iloc[0] - offense_B['Y']
    defense_B['Y_scaled'] = rusher_B['Y'].iloc[0] - defense_B['Y']
    
    
    # Initialize error counter
    error = 0
    # Match up and calculate error for offense
    while len(offense_A) > 0:
        curr = offense_A.iloc[0]
        play_error = 1000
        play_index = None
        for i in range(len(offense_B)):
            dist = math.sqrt((curr['X_scaled'] - offense_B['X_scaled'].iloc[i])**2 + 
                             (curr['Y_scaled'] - offense_B['Y_scaled'].iloc[i])**2)
            if dist < play_error:
                play_error = dist
                play_index = i
            
        error += play_error
        offense_B = offense_B.drop(offense_B.index[play_index])
        
        offense_A = offense_A.drop(offense_A.index[0])
    
    return error
    

In [103]:
calc_similar(A_id, B_id)

26.140444491203255

In [110]:
data = df[df['Yards'] >= 40]

In [111]:
len(data)

2442

In [112]:
plays_yards = data[['PlayId', 'Yards']].drop_duplicates()

from sklearn.model_selection import train_test_split
train , test = train_test_split(plays_yards, test_size = 0.3)

x_train = train.drop(['Yards'], axis=1)
y_train = train['Yards']

x_test = test.drop(['Yards'], axis = 1)
y_test = test['Yards']

In [113]:
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 

from sklearn.model_selection import GridSearchCV
params = {'n_neighbors': [*range(1, 5, 1)]}

knn = neighbors.KNeighborsRegressor(metric=calc_similar)

model = GridSearchCV(knn, params, cv=5)
model.fit(x_train,y_train)
model.best_params_

{'n_neighbors': 3}

In [114]:
model = neighbors.KNeighborsRegressor(n_neighbors = model.best_params_['n_neighbors'])
model.fit(x_train, y_train)  #fit the model
pred=model.predict(x_test) #make prediction on test set
pred_round = np.around(pred)
pred_round

array([55., 57., 60., 46., 48., 72., 53., 52., 53., 46., 53., 53., 57.,
       70., 55., 52., 53., 55., 46., 60., 44., 61., 56., 52., 51., 63.,
       53., 62., 54., 53., 46., 53., 53., 60.])

In [115]:
sample_sub = pd.read_csv('sample_submission.csv').rename(columns={"Unnamed: 0": "PlayId"}) #bring in sample sumbission
sub = sample_sub.set_index('PlayId').drop([0])
plays = pd.unique(test['PlayId'])

for i in range(plays.size):
    dist = np.histogram(np.random.normal(pred_round[i], 2, 1000), bins = 199, range = (-99, 99), density=True)[0] #find probability for each bin
    cumul_sum = np.cumsum(dist)
    sub.loc[plays[i]] = cumul_sum
sub = sub.reset_index()

In [116]:
score(sub, df)

0.039499259139804004

In [117]:
sub

,PlayId,Yards-99,Yards-98,Yards-97,Yards-96,Yards-95,Yards-94,Yards-93,Yards-92,Yards-91,Yards-90,Yards-89,Yards-88,Yards-87,Yards-86,Yards-85,Yards-84,Yards-83,Yards-82,Yards-81,Yards-80,Yards-79,Yards-78,Yards-77,Yards-76,Yards-75,Yards-74,Yards-73,Yards-72,Yards-71,Yards-70,Yards-69,Yards-68,Yards-67,Yards-66,Yards-65,Yards-64,Yards-63,Yards-62,Yards-61,Yards-60,Yards-59,Yards-58,Yards-57,Yards-56,Yards-55,Yards-54,Yards-53,Yards-52,Yards-51,Yards-50,Yards-49,Yards-48,Yards-47,Yards-46,Yards-45,Yards-44,Yards-43,Yards-42,Yards-41,Yards-40,Yards-39,Yards-38,Yards-37,Yards-36,Yards-35,Yards-34,Yards-33,Yards-32,Yards-31,Yards-30,Yards-29,Yards-28,Yards-27,Yards-26,Yards-25,Yards-24,Yards-23,Yards-22,Yards-21,Yards-20,Yards-19,Yards-18,Yards-17,Yards-16,Yards-15,Yards-14,Yards-13,Yards-12,Yards-11,Yards-10,Yards-9,Yards-8,Yards-7,Yards-6,Yards-5,Yards-4,Yards-3,Yards-2,Yards-1,Yards0,Yards1,Yards2,Yards3,Yards4,Yards5,Yards6,Yards7,Yards8,Yards9,Yards10,Yards11,Yards12,Yards13,Yards14,Yards15,Yards16,Yards17,Yards18,Yards19,Yards20,Yards21,Yards22,Yards23,Yards24,Yards25,Yards26,Yards27,Yards28,Yards29,Yards30,Yards31,Yards32,Yards33,Yards34,Yards35,Yards36,Yards37,Yards38,Yards39,Yards40,Yards41,Yards42,Yards43,Yards44,Yards45,Yards46,Yards47,Yards48,Yards49,Yards50,Yards51,Yards52,Yards53,Yards54,Yards55,Yards56,Yards57,Yards58,Yards59,Yards60,Yards61,Yards62,Yards63,Yards64,Yards65,Yards66,Yards67,Yards68,Yards69,Yards70,Yards71,Yards72,Yards73,Yards74,Yards75,Yards76,Yards77,Yards78,Yards79,Yards80,Yards81,Yards82,Yards83,Yards84,Yards85,Yards86,Yards87,Yards88,Yards89,Yards90,Yards91,Yards92,Yards93,Yards94,Yards95,Yards96,Yards97,Yards98,Yards99
0,20180923091488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004020,0.011056,0.031157,0.088444,0.188949,0.350763,0.551773,0.731677,0.867359,0.954798,0.988970,1.001030,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051
1,20171009002980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001005,0.001005,0.011056,0.030152,0.086434,0.178899,0.320611,0.529662,0.728662,0.871379,0.953793,0.993995,1.002035,1.004045,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051,1.005051
2,20181209060971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [119]:
test

,PlayId,Yards
301730,20180923091488,41
77528,20171009002980,58
453684,20181209060971,54
72358,20171008074020,90
502326,20181230082877,68
446226,20181203001224,90
371756,20181028063696,64
20284,20170917042617,53
358248,20181021062125,54
316976,20180930091111,63
